In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import time

In [2]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

In [3]:
from src.data_processing.downscaling import resample_image, Downscaler
from src.data_processing.sentinel_preprocessing import load_sentinel2_data

from utils.date_utils import (
    set_to_first_of_month,
    print_collection_dates,
    create_centered_date_ranges,
)
from utils.ee_utils import harmonized_ts, export_image_to_asset, back_to_int
from utils.harmonic_regressor import HarmonicRegressor
from utils.wapor_et_processing import load_wapor_et_data

from typing import List, Callable, Tuple

# 0: Define the region of interest

In [4]:
# PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"

# aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
# aoi_geometry = aoi_feature_collection.geometry().simplify(500)

# aoi = aoi_geometry.buffer(1000)

cantons = ee.FeatureCollection('projects/thurgau-irrigation/assets/GIS/Kantone_simplified100m')

aoi = cantons.filter(ee.Filter.eq('NAME', 'Solothurn')).geometry()

Map = geemap.Map(center=[47.4, 9.4], zoom=9)

Map.addLayer(aoi, {}, "AOI")
Map

Map(center=[47.4, 9.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

# 1: Exporting dekdal WaPOR ET data for Thurgau:

In [5]:
first_year = 2018
last_year = 2021
wapor_et_data = load_wapor_et_data(
    first_year, last_year, frequency="dekadal"
).filterBounds(aoi)

# Map = geemap.Map()

# image = ee.Image(wapor_et_data.toList(wapor_et_data.size()).get(20))

# Map.addLayer(image, {"min": 0, "max": 10}, "ET")
# Map.centerObject(aoi, 10)

# Map

In [6]:
SCALE = wapor_et_data.first().projection().nominalScale().getInfo()

print(f"The scale of the data is {SCALE}")

for year in range(first_year, last_year):
    et_wapor_year = wapor_et_data.filterDate(f"{year}-01-01", f"{year}-12-31")

    et_wapor_year_list = et_wapor_year.toList(et_wapor_year.size())

    for i in range(36):
        dekadal = i % 3 + 1
        month = i // 3 + 1

        image_to_export = ee.Image(et_wapor_year_list.get(i))

        task_name = f"ET_WAPOR_300m_{year}_{month}_D{dekadal}"
        asset_id = (
            f"projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/{task_name}"
        )

        export_image_to_asset(
            image=image_to_export,
            asset_id=asset_id,
            task_name=task_name,
            year=year,
            scale=SCALE,
            aoi=aoi,
        )

        print(f"Exported image {task_name} to asset {asset_id}")

    print(f"Done exporting year {year}, sleeping for 15 seconds")
    time.sleep(15)

print("Done exporting all images, mate!")

The scale of the data is 326.13132068341866
Exporting ET_WAPOR_300m_2018_1_D1 for 2018 to projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/ET_WAPOR_300m_2018_1_D1
Exported image ET_WAPOR_300m_2018_1_D1 to asset projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/ET_WAPOR_300m_2018_1_D1
Exporting ET_WAPOR_300m_2018_1_D2 for 2018 to projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/ET_WAPOR_300m_2018_1_D2
Exported image ET_WAPOR_300m_2018_1_D2 to asset projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/ET_WAPOR_300m_2018_1_D2
Exporting ET_WAPOR_300m_2018_1_D3 for 2018 to projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/ET_WAPOR_300m_2018_1_D3
Exported image ET_WAPOR_300m_2018_1_D3 to asset projects/thurgau-irrigation/assets/ET_products/Solothurn/ET_WaPOR_300m/ET_WAPOR_300m_2018_1_D3
Exporting ET_WAPOR_300m_2018_2_D1 for 2018 to projects/thurgau-irrigation/assets/ET_products/Solothurn/E

# 2: Exporting monthly landsat at 100m resolution

In [ ]:
# TODO: Once Silvan is done exporting to that collection

# Sanity check

In [ ]:
# Map = geemap.Map()

# image_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/ET_products/Thurgau/WaPOR_300m")
# image = ee.Image(image_collection.toList(image_collection.size()).get(20))

# Map.addLayer(image, {"min": 0, "max": 5, "palette": ["blue", "yellow", "red"]}, "ET")

# Map.centerObject(aoi, 10)

# Map



In [ ]:
# print_collection_dates(image_collection)